RESOURCES USED:
* https://www.stackabuse.com/how-to-iterate-over-rows-in-a-pandas-dataframe/
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1950838/

In [1]:
import pandas as pd
import numpy as np

In [2]:
# filteredlwkallsamples_fixed = TXT FILE THAT RESULTED FROM USING VCFTOOLS + BCFTOOLS ON THE PS2_IBD.LWK.VCF.GZ FILE 
#                               AND REMOVING SNPS WHERE THE MAF < 0.05 AND THE HWE < 0.001 AND THEN FORMATTING THE FILE
df = pd.read_csv('filteredlwkallsamples_fixed', sep = "\t", header = None)
print(df)

        0          1   2   3            4            5            6    \
0        10      98087   C   T  NA19020=0|0  NA19028=0|1  NA19035=0|0   
1        10     134767   A   G  NA19020=1|1  NA19028=0|0  NA19035=1|1   
2        10     135656   T   G  NA19020=0|0  NA19028=0|1  NA19035=0|0   
3        10     135708   G   A  NA19020=1|0  NA19028=0|0  NA19035=1|1   
4        10     135853   A   G  NA19020=1|1  NA19028=0|1  NA19035=1|1   
...     ...        ...  ..  ..          ...          ...          ...   
766754    9  141017240   C   T  NA19020=1|0  NA19028=0|0  NA19035=0|0   
766755    9  141017352   C   T  NA19020=1|0  NA19028=0|1  NA19035=1|0   
766756    9  141018423   C   T  NA19020=0|0  NA19028=0|0  NA19035=0|0   
766757    9  141025328   C   A  NA19020=0|0  NA19028=0|0  NA19035=0|0   
766758    9  141066491   G   A  NA19020=1|1  NA19028=0|1  NA19035=1|1   

                7            8            9    ...          91           92   \
0       NA19036=0|0  NA19038=0|1  NA19041=0

In [3]:
# CALCULATE THE NECESSARY VALUES AT EACH SNP THAT WILL LATER
# BE USED TO CALCULATE THE FINAL % IBD VALUES BETWEEN PAIRS OF SAMPLES

# DETERMINE X,Y,T_A,p,q FOR EACH SNP AND ADD TO AN ARRAY TO 
# APPEND TO THE DF 
# X = NUMBER OF '0' ALLELES AT THAT SNP ACROSS ALL SAMPLES
# Y = NUMBER OF '1' ALLELES AT THAT SNP ACROSS ALL SAMPLES
# T_A = TOTAL NUMBER OF ALLELES PRESENT AT THAT SNP ACROSS ALL SAMPLES
# p = X/T_A
# q = Y/T_A
X = []
Y = []
T_A = []
p = []
q = []

# CALCULATE NECESSARY P(IBS=i|IBD=z) PROBABILITIES AND ADD TO THE DATAFRAME
P_I0_Z0 = []
P_I1_Z0 = []
P_I1_Z1 = []
P_I2_Z0 = []
P_I2_Z1 = []
P_I2_Z2 = []

# FOR EACH SNP IN THE DATASET
for idx, row in df.iterrows():
    count_X = 0
    count_Y = 0
    count_T_A = 0
    # FOR EACH SAMPLE AT THAT CURRENT SNP 
    for i in range(4,101):
        # GET THE GENOTYPE OF THE CURRENT SAMPLE AT THAT SNP
        entry = row[i]
        entry_arr = entry.split('=')
        gen = entry_arr[1]
        allele_arr = gen.split('|')
        # ALLELE_1 = THE FIRST ALLELE IN THE GENOTYPE OF THE CURRENT SAMPLE AT THE GIVEN SNP
        # ALLELE_2 = THE SECOND ALLELE IN THE GENOTYPE OF THE CURRENT SAMPLE AT THE GIVEN SNP
        allele_1 = allele_arr[0]
        allele_2 = allele_arr[1]
        # ADD THE NUMBER OF '0' ALLELES FOUND IN THE CURRENT SAMPLE'S GENOTYPE AT THIS SNP 
        # TO THE TOTAL NUMBER OF '0' ALLELES FOUND AT THAT SNP SO FAR (count_X)
        # ADD THE NUMBER OF '1' ALLELES FOUND IN THE CURRENT SAMPLE'S GENOTYPE AT THIS SNP 
        # TO THE TOTAL NUMBER OF '1' ALLELES FOUND AT THAT SNP SO FAR (count_Y)
        # ADD THE NUMBER OF ALLELES PRESENT IN THE CURRENT SAMPLE'S GENOTYPE AT THIS SNP 
        # TO THE TOTAL NUMBER OF PRESENT ALLELES FOUND AT THAT SNP SO FAR (count_T_A)
        if (allele_1 == '0'):
            count_X += 1
            count_T_A += 1
        if (allele_2 == '0'):
            count_X += 1
            count_T_A += 1
        if (allele_1 == '1'):
            count_Y += 1
            count_T_A += 1
        if (allele_2 == '1'):
            count_Y += 1
            count_T_A += 1
    # ADD THE INDIVIDUAL VALUES CALCULATED AT EACH SNP TO AN ARRAY TO LATER APPEND TO THE DATAFRAME
    X.append(count_X)
    Y.append(count_Y)
    T_A.append(count_T_A)
    p_val = count_X/count_T_A
    q_val = count_Y/count_T_A
    p.append(p_val)
    q.append(q_val) 
    
    # AT EACH SNP, USED THE VALUES CALCULATED ABOVE IN ORDER TO 
    # CALCULATE THE 6 NECESSARY PROBABILITIES THAT INVOLVE P(IBS=i|IBD=z)
    # FOR THAT SNP.
    # FORMULAS CAN BE FOUND IN THE PLINK PAPER 
    # 1. P(IBS=0|IBD=0)
    # CHECK TO MAKE SURE THAT THE DENOMINATOR IS GREATER THAN 0, 
    # OTHERWISE SET THAT ENTIRE TERM TO 0 
    if (count_X > 0):
        term1 = (count_X-1)/count_X
    else:
        term1 = 0
    if (count_Y > 0):
        term2 = (count_Y-1)/count_Y
    else:
        term2 = 0
    term3 = count_T_A/(count_T_A-1)
    term4 = count_T_A/(count_T_A-2)
    term5 = count_T_A/(count_T_A-3)
    P_IBS0_GIVEN_IBD0 = 2*(p_val**2)*(q_val**2)*(term1*term2*term3*term4*term5)
    P_I0_Z0.append(P_IBS0_GIVEN_IBD0)
    # 2. P(IBS=1|IBD=0)
    # CHECK TO MAKE SURE THAT THE DENOMINATOR IS GREATER THAN 0, 
    # OTHERWISE SET THAT ENTIRE TERM TO 0 
    if (count_X > 0):
        term1 = (count_X-1)/count_X
        term2 = (count_X-2)/count_X
    else:
        term1 = 0
        term2 = 0
    term3 = count_T_A/(count_T_A-1)
    term4 = count_T_A/(count_T_A-2)
    term5 = count_T_A/(count_T_A-3)
    if (count_Y > 0):
        term6 = (count_Y-1)/count_Y
        term7 = (count_Y-2)/count_Y
    else:
        term6 = 0
        term7 = 0
    coeff1 = 4*(p_val**3)*(q_val)
    coeff2 = 4*(p_val)*(q_val**3)
    first_part = coeff1 * (term1*term2*term3*term4*term5)
    second_part = coeff2 * (term6*term7*term3*term4*term5)
    P_IBS1_GIVEN_IBD0 = first_part + second_part
    P_I1_Z0.append(P_IBS1_GIVEN_IBD0)
    # 3. P(IBS=1|IBD=1)
    # CHECK TO MAKE SURE THAT THE DENOMINATOR IS GREATER THAN 0, 
    # OTHERWISE SET THAT ENTIRE TERM TO 0 
    if (count_X > 0):
        term1 = (count_X-1)/count_X
    else:
        term1 = 0
    term2 = count_T_A/(count_T_A-1)
    term3 = count_T_A/(count_T_A-2)
    if (count_Y > 0):
        term4 = (count_Y-1)/count_Y
    else:
        term4 = 0
    coeff1 = 2*(p_val**2)*(q_val)
    coeff2 = 2*(p_val)*(q_val**2)
    first_part = coeff1 * (term1*term2*term3)
    second_part = coeff2 * (term4*term2*term3)
    P_IBS1_GIVEN_IBD1 = first_part + second_part
    P_I1_Z1.append(P_IBS1_GIVEN_IBD1)    
    # 4. P(IBS=2|IBD=0)
    # CHECK TO MAKE SURE THAT THE DENOMINATOR IS GREATER THAN 0, 
    # OTHERWISE SET THAT ENTIRE TERM TO 0 
    coeff1 = p_val**4
    coeff2 = q_val**4
    coeff3 = 4*(p_val**2)*(q_val**2)
    if (count_X > 0):
        term1 = (count_X-1)/count_X
        term2 = (count_X-2)/count_X
        term3 = (count_X-3)/count_X
    else:
        term1 = 0
        term2 = 0
        term3 = 0
    term4 = count_T_A/(count_T_A-1)
    term5 = count_T_A/(count_T_A-2)
    term6 = count_T_A/(count_T_A-3)
    if (count_Y > 0):
        term7 = (count_Y-1)/count_Y
        term8 = (count_Y-2)/count_Y
        term9 = (count_Y-3)/count_Y
    else:
        term7 = 0
        term8 = 0
        term9 = 0
    first_part = coeff1 * (term1*term2*term3*term4*term5*term6)
    second_part = coeff2 * (term7*term8*term9*term4*term5*term6)
    third_part = coeff3 * (term1*term7*term4*term5*term6)
    P_IBS2_GIVEN_IBD0 = first_part + second_part + third_part
    P_I2_Z0.append(P_IBS2_GIVEN_IBD0)
    # 5. P(IBS=2|IBD=1)
    # CHECK TO MAKE SURE THAT THE DENOMINATOR IS GREATER THAN 0, 
    # OTHERWISE SET THAT ENTIRE TERM TO 0 
    coeff1 = p_val**3
    coeff2 = q_val**3
    coeff3 = (p_val**2) * q_val
    coeff4 = p_val * (q_val**2)
    if (count_X > 0):
        term1 = (count_X-1)/count_X
        term2 = (count_X-2)/count_X
    else:
        term1 = 0
        term2 = 0
    term3 = count_T_A/(count_T_A-1)
    term4 = count_T_A/(count_T_A-2)
    if (count_Y > 0):
        term5 = (count_Y-1)/count_Y
        term6 = (count_Y-2)/count_Y
    else:
        term5 = 0
        term6 = 0
    first_part = coeff1 * (term1*term2*term3*term4)
    second_part = coeff2 * (term5*term6*term3*term4)
    third_part = coeff3 * (term1*term3*term4)
    fourth_part = coeff4 * (term5*term3*term4)
    P_IBS2_GIVEN_IBD1 = first_part + second_part + third_part + fourth_part
    P_I2_Z1.append(P_IBS2_GIVEN_IBD1)
    # 6. P(IBS=2|IBD=2)
    P_IBS2_GIVEN_IBD2 = 1
    P_I2_Z2.append(P_IBS2_GIVEN_IBD2)

# ADDING ALL OF THE ARRAYS OF VALUES CALCULATED ABOVE TO THE INITIAL DATAFRAME
df['X'] = X
df['Y'] = Y
df['T_A'] = T_A
df['p'] = p
df['q'] = q

df['P(IBS=0|IBD=0)'] = P_I0_Z0
df['P(IBS=1|IBD=0)'] = P_I1_Z0
df['P(IBS=1|IBD=1)'] = P_I1_Z1
df['P(IBS=2|IBD=0)'] = P_I2_Z0
df['P(IBS=2|IBD=1)'] = P_I2_Z1
df['P(IBS=2|IBD=2)'] = P_I2_Z2        

In [4]:
# CHECKING TO SEE WHAT THE DATAFRAME LOOKS LIKE NOW THAT 
# VALUES HAVE BEEN APPENDED TO IT
df

,0,1,2,3,4,5,6,7,8,9,...,Y,T_A,p,q,P(IBS=0|IBD=0),P(IBS=1|IBD=0),P(IBS=1|IBD=1),P(IBS=2|IBD=0),P(IBS=2|IBD=1),P(IBS=2|IBD=2)
0,10,98087,C,T,NA19020=0|0,NA19028=0|1,NA19035=0|0,NA19036=0|0,NA19038=0|1,NA19041=0|0,...,18,194,0.907216,0.092784,0.013728,0.283531,0.169222,0.702741,0.830778,1
1,10,134767,A,G,NA19020=1|1,NA19028=0|0,NA19035=1|1,NA19036=1|1,NA19038=1|0,NA19041=1|1,...,124,194,0.360825,0.639175,0.107303,0.498090,0.463650,0.394607,0.536350,1
2,10,135656,T,G,NA19020=0|0,NA19028=0|1,NA19035=0|0,NA19036=0|0,NA19038=0|1,NA19041=0|0,...,19,194,0.902062,0.097938,0.015169,0.294541,0.177608,0.690290,0.822392,1
3,10,135708,G,A,NA19020=1|0,NA19028=0|0,NA19035=1|1,NA19036=1|1,NA19038=1|0,NA19041=1|1,...,125,194,0.355670,0.644330,0.105932,0.497698,0.460713,0.396370,0.539287,1
4,10,135853,A,G,NA19020=1|1,NA19028=0|1,NA19035=1|1,NA19036=1|1,NA19038=1|1,NA19041=1|1,...,146,194,0.247423,0.752577,0.069566,0.470414,0.374339,0.460020,0.625661,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766754,9,141017240,C,T,NA19020=1|0,NA19028=0|0,NA19035=0|0,NA19036=0|0,NA19038=0|0,NA19041=0|0,...,14,194,0.927835,0.072165,0.008541,0.235050,0.134608,0.756408,0.865392,1
766755,9,141017352,C,T,NA19020=1|0,NA19028=0|1,NA19035=1|0,NA19036=1|1,NA19038=1|1,NA19041=1|0,...,164,194,0.154639,0.845361,0.033876,0.390111,0.262806,0.576014,0.737194,1
766756,9,141018423,C,T,NA19020=0|0,NA19028=0|0,NA19035=0|0,NA19036=1|0,NA19038=0|0,NA19041=0|0,...,41,194,0.788660,0.211340,0.055554,0.447941,0.335078,0.496505,0.664922,1
766757,9,141025328,C,A,NA19020=0|0,NA19028=0|0,NA19035=0|0,NA19036=0|1,NA19038=0|0,NA19041=0|0,...,45,194,0.768041,0.231959,0.063599,0.461912,0.358154,0.474489,0.641846,1


In [5]:
# SUM TOGETHER ALL OF THE CORRESPONDING INDIVIDUAL P(IBS=i|IBD=z) PROBABILITIES 
# FOUND AT EACH SNP IN ORDER TO GET THE 6 OVERALL N(IBS=i|IBD=z) PROBABILITIES 
# THAT WE NEED IN ORDER TO DO FINAL % IBD CALCULATIONS
N_I0_Z0 = df['P(IBS=0|IBD=0)'].sum()    
N_I1_Z0 = df['P(IBS=1|IBD=0)'].sum()  
N_I1_Z1 = df['P(IBS=1|IBD=1)'].sum()
N_I2_Z0 = df['P(IBS=2|IBD=0)'].sum()
N_I2_Z1 = df['P(IBS=2|IBD=1)'].sum()
N_I2_Z2 = df['P(IBS=2|IBD=2)'].sum()

In [ ]:
# GOAL: CALCULATE THE FINAL % IBD VALUES FOR EACH PAIR OF SAMPLES IN THE DATASET
#       AND WRITE THE RESULTS TO AN OUTPUT FILE 

# NESTED FOR LOOPS WILL ALLOWS US TO ITERATE OVER ALL PAIRS OF SAMPLES IN THE DATAFRAME
# FOR EACH SAMPLE 1 IN THE DATASET 
for sample_1 in range(4, 100):
    # GET THE INDEX OF THE FIRST SAMPLE
    sample1_idx = sample_1
    # FOR EACH SAMPLE 2 THAT IS AFTER THE CURRENT SAMPLE 1
    # ESSENTIALLY MEANS, WE DON'T HAVE % IBD VALUES FOR THAT PAIR OF SAMPLES IN OUTPUT FILE YET
    for sample_2 in range(sample_1+1, 101):
        # GET THE INDEX OF THE SECOND SAMPLE
        sample2_idx = sample_2
       
        # KEEPING TRACK OF NUMBER OF CORRESPONDING SNPS THAT HAVE IBS = 0,1,2 BETWEEN SAMPLE 1 AND SAMPLE 2 
        # OF THE PAIR OF SAMPLES WE ARE CURRENTLY COMPARING
        IBS0_COUNT = 0
        IBS1_COUNT = 0
        IBS2_COUNT = 0
        
        sample1_name = ""
        sample2_name = ""
        # FOR EVERY SNP:
        for idx, row in df.iterrows():
            #SPLIT UP THE SAMPLE 1 GENOTYPE INTO ALLELES FIRST
            sample1_GT = row[sample1_idx]

            gt_arr =  sample1_GT.split('=')
            # GETTING NAME FOR THE SAMPLE 
            sample1_name = gt_arr[0]
            # GETTING GENOTYPE FOR THE SAMPLE
            gt_1 = gt_arr[1]

            allele_arr = gt_1.split("|")
            # SAMPLE1_ALLELE_1 = THE FIRST ALLELE IN THE GENOTYPE OF THE SAMPLE 1 AT THE GIVEN SNP
            # SAMPLE1_ALLELE_2 = THE SECOND ALLELE IN THE GENOTYPE OF THE SAMPLE 1 AT THE GIVEN SNP
            sample1_allele_1 = allele_arr[0]
            sample1_allele_2 = allele_arr[1]

            # SPLIT UP THE SAMPLE 2 GENOTYPE INTO ALLELES 
            sample2_GT = row[sample2_idx]

            gt_arr = sample2_GT.split('=')
            # GETTING NAME FOR THE SAMPLE
            sample2_name = gt_arr[0]
            # GETTING GENOTYPE FOR THE SAMPLE
            gt_1 = gt_arr[1]

            allele_arr = gt_1.split("|")
            # SAMPLE2_ALLELE_1 = THE FIRST ALLELE IN THE GENOTYPE OF THE SAMPLE 2 AT THE GIVEN SNP
            # SAMPLE2_ALLELE_2 = THE SECOND ALLELE IN THE GENOTYPE OF THE SAMPLE 2 AT THE GIVEN SNP
            sample2_allele_1 = allele_arr[0]
            sample2_allele_2 = allele_arr[1]

            # DETERMINE IBS BETWEEN THE TWO SAMPLES AT THIS SNP
            # COUNT HOW MANY X (ALLELES = '0') AND Y (ALLELES = '1') IN THE GENOTYPES FOR SAMPLE 1
            # AND SAMPLE 2 INDIVIDUALLY
            x_sample1 = 0
            y_sample1 = 0
            if (sample1_allele_1 == '0'):
                x_sample1 += 1
            if (sample1_allele_2 == '0'):
                x_sample1 += 1
            if (sample1_allele_1 == '1'):
                y_sample1 += 1
            if (sample1_allele_2 == '1'):
                y_sample1 += 1
            x_sample2 = 0
            y_sample2 = 0
            if (sample2_allele_1 == '0'):
                x_sample2 += 1
            if (sample2_allele_2 == '0'):
                x_sample2 += 1
            if (sample2_allele_1 == '1'):
                y_sample2 += 1
            if (sample2_allele_2 == '1'):
                y_sample2 += 1
            # DETERMINE THE IBS LEVEL BETWEEN THE TWO SAMPLES IN THIS PAIR AT THIS SNP AND
            # INCREMENT THE COUNT OF THE CORRESPONDING IBS LEVEL (IBS0,IBS1,OR IBS2) 
            if ((x_sample1 ==  x_sample2) and (y_sample1 == y_sample2)):
                IBS2_COUNT += 1
            elif ((x_sample1 == y_sample2) and (x_sample2 == y_sample1)):
                IBS0_COUNT += 1
            else:
                IBS1_COUNT += 1
        # FINAL IBD = 0, 1, 2 CALCULATIONS 
        # USE THE VALUES CALCULATED PRIOR (IN THIS CODING CELL AND THE ONE ABOVE IT) AND PLUG THEM 
        # INTO THE FORMULAS FOR THE P(IBD=0), P(IBD=1), AND P(IBD=2) CALCULATIONS FROM THE PLINK PAPER
        PROB_IBD_0 = IBS0_COUNT/N_I0_Z0
        PROB_IBD_1 = (IBS1_COUNT-(PROB_IBD_0*N_I1_Z0))/N_I1_Z1
        PROB_IBD_2 = (IBS2_COUNT-(PROB_IBD_0*N_I2_Z0)-(PROB_IBD_1*N_I2_Z1))/N_I2_Z2
        # BOUND THE PROBABILITY P(IBD=0) AND IN TURN P(IBD=1) BETWEEN 0 AND 1 
        # BASED OFF THE RULES/FORMULAS DESCRIBED IN THE PLINK PAPER FOR RECALCULATING
        # THE IBD=0,1,AND 2 PROBABILITIES 
        pi = (PROB_IBD_1/2) + PROB_IBD_2 
        pi_sq = pi * pi 
        # IF THE P(IBD=0) WAS INITIALLY OVER 1, THEN SET THE P(IBD=0) TO 1 
        # AND THE P(IBD=1) AND P(IBD=2) TO 0
        if (PROB_IBD_0 > 1):
            PROB_IBD_0 = 1 
            PROB_IBD_1 = 0
            PROB_IBD_2 = 0
        # IF THE P(IBD=0) WAS INITIALLY LESS THAN 0, THEN SET THE P(IBD=0) TO 0
        # AND SET P(IBD=1) TO THE INITIAL P(IBD=1) DIVIDED BY P(IBD=1)+P(IBD=2)
        # AND SET P(IBD=2) TO THE INITIAL P(IBD=2) DIVIDED BY P(IBD=1)+P(IBD=2)
        elif (PROB_IBD_0 < 0):
            PROB_IBD_0 = 0
            S = PROB_IBD_1 + PROB_IBD_2 
            PROB_IBD_1 = PROB_IBD_1/S
            PROB_IBD_2 = PROB_IBD_2/S
        # ANOTHER POSSIBILITY, IF P(IBD=1) IS VERY NEGATIVE OR 0 THEN SET 
        # PROB(IBD=0) TO (1-(P(IBD=1)/2 + P(IBD=2)))^2
        # AND P(IBD=1) TO 2 * (P(IBD=1)/2 + P(IBD=2)) * (1-(P(IBD=1)/2 + P(IBD=2)))
        # AND P(IBD=2) TO (P(IBD=1)/2 + P(IBD=2))^2
        elif (pi_sq <= PROB_IBD_2):
            PROB_IBD_0 = (1-pi)*(1-pi)
            PROB_IBD_1 = 2*pi*(1-pi)
            PROB_IBD_2 = pi_sq
        # FORMAT + WRITE THE P(IBD=0), P(IBD=1), AND P(IBD=2) VALUES 
        # FOR THIS PAIR OF SAMPLES TO THE OUTPUT RESULTS FILE
        output = open('FINAL_RESULTS_FILTERED_AND_BOUNDED_FIXEDFORPRESENTATION','a')
        stringtoprint = sample1_name+"\t"+sample2_name+"\t"+str(PROB_IBD_0)+"\t"+str(PROB_IBD_1)+"\t"+str(PROB_IBD_2)+"\n"
        output.write(stringtoprint)
        output.close()